In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
dataset_path = "/content/gdrive/My Drive/datasets/tas-nlp/"

In [3]:
import re

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

# TensorFlow and tf.keras
import tensorflow as tf
from tensorflow import keras

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical

print('Keras       ', keras.__version__)
print('TensorFlow  ', tf.__version__)

Keras        2.2.4-tf
TensorFlow   1.13.1


Using TensorFlow backend.


In [4]:

from tensorflow.python.client import device_lib

def get_available_gpus():
    local_device_protos = device_lib.list_local_devices()
    return [x.name for x in local_device_protos if x.device_type == 'GPU']

get_available_gpus()

['/device:GPU:0']

In [0]:
# drive.mount("/content/gdrive", force_remount=True)

In [6]:
! ls -la /content/gdrive/'My Drive'/datasets/tas-nlp/prolexitim-merged-1.3.csv
      

-rw------- 1 root root 209682 May 13 09:39 '/content/gdrive/My Drive/datasets/tas-nlp/prolexitim-merged-1.3.csv'


In [0]:
tasnlp_df = pd.read_csv(dataset_path + "prolexitim-merged-1.3.csv", header=0, delimiter="\t")

In [0]:
# tasnlp_df.head()

In [0]:
# We only need text and label
data = tasnlp_df[['Text','alex-a']]

In [10]:
data.head()

,Text,alex-a
0,Era un niño pensando en el granero pensando a ...,NoAlex
1,"Una madre que está consolando a su hijo, despu...",NoAlex
2,Un pantanal con una barca abandonada. A ver qu...,NoAlex
3,"Era un paraje muy bonito, con una barca, un po...",Alex
4,"Era una vez un matrimonio, que se quería muchí...",Alex


In [11]:
print(data.dtypes)
print(data.count())

Text      object
alex-a    object
dtype: object
Text      334
alex-a    318
dtype: int64


In [0]:
data = data.dropna()

In [13]:
data.count()

Text      318
alex-a    318
dtype: int64

In [14]:
data.groupby('alex-a').count()

,Text
alex-a,
Alex,31
NoAlex,242
PosAlex,45


In [0]:
# Join alexithymia and possible alexithymia as one positive class
data['alex-a'] = data['alex-a'].apply(lambda x: x.replace('PosAlex', 'Alex'))

In [16]:
data.groupby('alex-a').count()

,Text
alex-a,
Alex,76
NoAlex,242


In [0]:
# Cleanining text

import unicodedata
def strip_accents(s):
   return ''.join(c for c in unicodedata.normalize('NFD', s)
                  if unicodedata.category(c) != 'Mn')

def remove_accents(input_str):
    nfkd_form = unicodedata.normalize('NFKD', input_str)
    return u"".join([c for c in nfkd_form if not unicodedata.combining(c)])
                    

In [0]:
# Cleanining text

data['Text'] = data['Text'].apply(lambda x: remove_accents(x))
data['Text'] = data['Text'].apply(lambda x: x.lower())
data['Text'] = data['Text'].apply((lambda x: re.sub('[^ña-zA-z0-9\s]', '', x)))

In [19]:
data.tail(n=4)

,Text,alex-a
330,el nino estaba sangrando por la boca porque se...,Alex
331,se habian pasado todo el dia corriendo y jugan...,Alex
332,hay unos toros que estan enfadados y se prepar...,Alex
333,un senor que se arrepiente de haber matado a s...,Alex


In [20]:
nb_alex = data[data['alex-a'] == 'Alex'].size
nb_no_alex = data[data['alex-a'] == 'NoAlex'].size


print(f"There are {nb_alex} positive intances of alexithymia or possible alexithymia.")
print(f"There are {nb_no_alex} negative instances, no alexithymia detected")


There are 152 positive intances of alexithymia or possible alexithymia.
There are 484 negative instances, no alexithymia detected


In [0]:
# Keras vectorization of the text corpus 
max_features = 2000

tokenizer = Tokenizer(num_words=max_features, split=' ')
tokenizer.fit_on_texts(data['Text'].values)

X = tokenizer.texts_to_sequences(data['Text'].values)

In [22]:
print(f"Text: {data['Text'][1]}. Vector: {X[1]}") 
print(f"Text: {data['Text'][2]}. Vector: {X[2]}") 

Text: una madre que esta consolando a su hijo despues de darle las notas que ha sacado no se cuantos insuficientes. Vector: [8, 160, 1, 16, 870, 5, 11, 125, 45, 2, 871, 18, 397, 1, 38, 872, 12, 10, 539, 873]
Text: un pantanal con una barca abandonada a ver que estan haciendo los de la barca buscar setas. Vector: [4, 874, 14, 8, 161, 540, 5, 112, 1, 39, 126, 19, 2, 7, 161, 212, 875]


In [0]:
# Make sure we have the same length in all document vector
X = pad_sequences(X)

In [24]:
print(f"Vector Length: {len(X[1])}")
print(X[1])
print(f"Vector Length: {len(X[131])}")
print(X[131])
print(f"Vector Length: {len(X[3])}")
print(X[3])

Vector Length: 162
[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   8 160
   1  16 870   5  11 125  45   2 871  18 397   1  38 872  12  10 539 873]
Vector Length: 162
[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0

In [0]:
# One HOT encoding for alexithymia label
# and the values in Y. 

Y = pd.get_dummies(data['alex-a']).values

In [43]:
# Train / Test partitions (90/10)

X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.10, random_state = 42)

print(f"Train shape. Feature Vector:{X_train.shape}, Label:{Y_train.shape} (binary classification)")
print(f"Test shape. Feature Vector: {X_test.shape}, Label:{Y_test.shape} (binary classification)")

Train shape. Feature Vector:(286, 162), Label:(286, 2) (binary classification)
Test shape. Feature Vector: (32, 162), Label:(32, 2) (binary classification)


In [34]:
# Deep RNN Model Definition:
# Embedding -> Dropout -> LSTM -> Dense 

embed_dim = 128
lstm_out = 196

model = Sequential()
model.add(Embedding(max_features, embed_dim, input_length = X.shape[1]))
model.add(SpatialDropout1D(0.4))
model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(2,activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model.summary())

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 162, 128)          256000    
_________________________________________________________________
spatial_dropout1d_1 (Spatial (None, 162, 128)          0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 196)               254800    
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 394       
Total params: 511,194
Trainable params: 511,194
Non-trainable params: 0
_________________________________________________________________
None


In [35]:
# Train the model
batch_size = 32
model.fit(X_train, Y_train, epochs = 15, batch_size=batch_size, verbose = 2)

Instructions for updating:
Use tf.cast instead.
Epoch 1/15
 - 4s - loss: 0.6469 - acc: 0.7273
Epoch 2/15
 - 3s - loss: 0.5807 - acc: 0.7622
Epoch 3/15
 - 2s - loss: 0.5310 - acc: 0.7622
Epoch 4/15
 - 2s - loss: 0.5030 - acc: 0.7622
Epoch 5/15
 - 2s - loss: 0.4540 - acc: 0.7622
Epoch 6/15
 - 2s - loss: 0.3671 - acc: 0.7762
Epoch 7/15
 - 2s - loss: 0.3381 - acc: 0.9056
Epoch 8/15
 - 2s - loss: 0.1956 - acc: 0.9441
Epoch 9/15
 - 2s - loss: 0.1706 - acc: 0.9685
Epoch 10/15
 - 2s - loss: 0.0878 - acc: 0.9720
Epoch 11/15
 - 2s - loss: 0.0633 - acc: 0.9825
Epoch 12/15
 - 2s - loss: 0.0320 - acc: 1.0000
Epoch 13/15
 - 2s - loss: 0.0324 - acc: 0.9965
Epoch 14/15
 - 2s - loss: 0.0244 - acc: 0.9930
Epoch 15/15
 - 2s - loss: 0.0162 - acc: 1.0000


In [42]:
print(X_test.shape)
print(Y_test.shape)
print(f"Test shape. Feature Vector: {X_test.shape}, Label:{Y_test.shape} (binary classification)")

(0, 162)
(0, 2)
Test shape. Feature Vector: (0, 162), Label:(0, 2) (binary classification)


In [44]:
# validation_size = 2

# X_validate = X_test[-validation_size:]
# Y_validate = Y_test[-validation_size:]
# X_test = X_test[:-validation_size]
# Y_test = Y_test[:-validation_size]

score,acc = model.evaluate(X_test, Y_test, verbose = 2, batch_size = batch_size)

print(model.metrics_names)
print("score: %.2f" % (score))
print("acc: %.2f" % (acc))

['loss', 'acc']
score: 0.62
acc: 0.75


In [84]:
narratives = [' un niño al que sus padres de clase acomodada le dijeron que tenía que aprender a tocar el violín y el niño lo odiaba.', 
             'Hay unos toros que están enfadados y se preparan para atacar. Van a bajar todos corriendo por la cuesta y si pillan a alguien lo más seguro que es que lo maten.',
             'Varios hombres se encuentran echando una siesta en su pausa para el almuerzo. Bajo un sol y calor de justicia, la parada se agradece tremendamente. Las horas de trabajo físico en el campo se hacen largas y cansadas para ganar un jornal que a duras penas les permite llegar a final de mes. Sólo esperan coger fuerzas durante el descanso para terminar la jornada lo antes posible.',
             'un hombre que mata a su amante porque ella le amenazó con contárselo a su mujer si este no se iba con ella..',
             'tanto me han apretado con lo de las clases extraescolares, que he desarrollado el superpoder de tocar el violín con la mente, mientras los demás compañeros se dejan el cuello sujetando el violín, yo lo toco mandándole órdenes cerebrales al instrumento, el primer día que lo conseguí el profesor me dió una colleja pensando que dormía, cuando empezó a sonar se asustó tanto que ya ni se me acerca, ahora aprovecho las clases de violín para tocar con la mente y para dormir a intervalos con la certeza de que la mente del profesor no va a decirle a su mano que me ajusticie. Cada uno a lo suyo. Fin',
             'Se levantó y vio que nada podía ser más bello y nadie podría contemplarlo igual']
p_narratives =[]

for text in narratives:
  p_narratives.append(remove_accents(text).lower())
  

print(p_narratives)

[' un nino al que sus padres de clase acomodada le dijeron que tenia que aprender a tocar el violin y el nino lo odiaba.', 'hay unos toros que estan enfadados y se preparan para atacar. van a bajar todos corriendo por la cuesta y si pillan a alguien lo mas seguro que es que lo maten.', 'varios hombres se encuentran echando una siesta en su pausa para el almuerzo. bajo un sol y calor de justicia, la parada se agradece tremendamente. las horas de trabajo fisico en el campo se hacen largas y cansadas para ganar un jornal que a duras penas les permite llegar a final de mes. solo esperan coger fuerzas durante el descanso para terminar la jornada lo antes posible.', 'un hombre que mata a su amante porque ella le amenazo con contarselo a su mujer si este no se iba con ella..', 'tanto me han apretado con lo de las clases extraescolares, que he desarrollado el superpoder de tocar el violin con la mente, mientras los demas companeros se dejan el cuello sujetando el violin, yo lo toco mandandole 

In [85]:
# vectorizing the text by the pre-fitted tokenizer instance

pp_narratives = tokenizer.texts_to_sequences(p_narratives)

print(pp_narratives)

[[4, 22, 23, 1, 29, 53, 2, 241, 1389, 13, 406, 1, 58, 1, 93, 5, 40, 6, 21, 3, 6, 22, 20, 1390], [65, 42, 1, 39, 3, 10, 17, 563, 5, 472, 69, 265, 15, 7, 3, 50, 5, 303, 20, 41, 326, 1, 27, 1, 20], [252, 121, 10, 731, 417, 8, 81, 9, 11, 17, 6, 234, 4, 75, 3, 609, 2, 1827, 7, 1171, 10, 802, 18, 124, 2, 74, 9, 6, 76, 10, 966, 1274, 3, 17, 4, 1979, 1, 5, 102, 139, 5, 466, 2, 85, 773, 669, 155, 6, 236, 17, 7, 201, 20, 105], [4, 30, 1, 725, 5, 11, 283, 28, 56, 13, 726, 14, 1397, 5, 11, 37, 50, 97, 12, 10, 172, 14, 56], [106, 54, 44, 14, 20, 2, 18, 103, 1, 583, 6, 2, 40, 6, 21, 14, 7, 534, 66, 19, 473, 204, 10, 6, 627, 6, 21, 230, 20, 840, 23, 135, 6, 34, 1, 20, 6, 294, 54, 189, 8, 95, 1, 373, 52, 401, 5, 383, 10, 106, 1, 73, 174, 10, 54, 94, 496, 18, 103, 2, 21, 17, 40, 14, 7, 534, 3, 17, 159, 5, 14, 7, 2, 1, 7, 534, 24, 294, 12, 164, 5, 5, 11, 850, 1, 54, 90, 175, 5, 20, 167], [10, 367, 3, 228, 1, 79, 111, 48, 41, 3, 213, 114, 636]]


In [86]:
# Padding the tweet to have exactly the same shape as `embedding_2` input

pp_narratives = pad_sequences(pp_narratives, maxlen=162, dtype='int32', value=0)
print(pp_narratives)


[[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    4   22
    23    1   29   53    2  241 1389   13  406    1   58    1   93    5
    40    6   21    3    6   22   20 1390]
 [   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0

In [87]:
# Testing the predictive model
alex_screening = model.predict(pp_narratives, batch_size=1, verbose = 2)
text_idx = 0 
for prediction in alex_screening:
  # print(prediction)
  print("NARRATIVE: " + narratives[text_idx])
  text_idx += 1
  if(np.argmax(prediction) == 0):
      print("PREDICTION: negative: No Alexithymia.")
  elif (np.argmax(prediction) == 1):
    print("PREDICTION: positive: Possible Alexithymia.")
  print("-----------------")

NARRATIVE:  un niño al que sus padres de clase acomodada le dijeron que tenía que aprender a tocar el violín y el niño lo odiaba.
PREDICTION: negative: No Alexithymia.
-----------------
NARRATIVE: Hay unos toros que están enfadados y se preparan para atacar. Van a bajar todos corriendo por la cuesta y si pillan a alguien lo más seguro que es que lo maten.
PREDICTION: negative: No Alexithymia.
-----------------
NARRATIVE: Varios hombres se encuentran echando una siesta en su pausa para el almuerzo. Bajo un sol y calor de justicia, la parada se agradece tremendamente. Las horas de trabajo físico en el campo se hacen largas y cansadas para ganar un jornal que a duras penas les permite llegar a final de mes. Sólo esperan coger fuerzas durante el descanso para terminar la jornada lo antes posible.
PREDICTION: positive: Possible Alexithymia.
-----------------
NARRATIVE: un hombre que mata a su amante porque ella le amenazó con contárselo a su mujer si este no se iba con ella..
PREDICTION: ne